<a href="https://colab.research.google.com/github/Aqillaaprly/rakamin_homework/blob/main/exam_week_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, classification_report, confusion_matrix
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/rakamin/diabetes.csv'
df = pd.read_csv(path)

In [ ]:
print(df.head())

   num_pregnant  glucose_concentration  blood_pressure  triceps_thickness  \
0             6                    148              72                 35   
1             1                     85              66                 29   
2             8                    183              64                  0   
3             1                     89              66                 23   
4             0                    137              40                 35   

   two_hour_insulin   bmi  pedigree_function  age  is_diabetes  
0                 0  33.6              0.627   50            1  
1                 0  26.6              0.351   31            0  
2                 0  23.3              0.672   32            1  
3                94  28.1              0.167   21            0  
4               168  43.1              2.288   33            1  


In [ ]:
# 3. Pisahkan Fitur dan Target
# =====================================
X = df.drop('is_diabetes', axis=1)
y = df['is_diabetes']


In [ ]:
# 4. Split Data Train dan Test
# =====================================
# Rasio 70:30, random_state=42 agar hasil konsisten
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print("Jumlah data train:", X_train.shape[0])
print("Jumlah data test:", X_test.shape[0])

Jumlah data train: 537
Jumlah data test: 231


In [ ]:
# 5. Model Random Forest (tanpa tuning)
# =====================================
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

In [ ]:
print("\n--- Evaluasi Model Random Forest ---")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))



--- Evaluasi Model Random Forest ---
              precision    recall  f1-score   support

           0       0.82      0.80      0.81       151
           1       0.64      0.66      0.65        80

    accuracy                           0.75       231
   macro avg       0.73      0.73      0.73       231
weighted avg       0.76      0.75      0.75       231

Confusion Matrix:
 [[121  30]
 [ 27  53]]
Recall: 0.6625


In [ ]:
# --- 4. Inisialisasi model dasar ---
rf = RandomForestClassifier(random_state=42)

# --- 5. Definisi grid hyperparameter sesuai soal ---
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_features': ['auto', 'sqrt', 'log2']
}

# --- 6. Grid search dengan 5-fold cross validation ---
grid = GridSearchCV(estimator=rf,
                    param_grid=param_grid,
                    cv=5,
                    scoring='recall',
                    n_jobs=-1)

grid.fit(X_train, y_train)


# --- 7. Evaluasi model terbaik ---
best_rf = grid.best_estimator_
y_pred = best_rf.predict(X_test)
recall = recall_score(y_test, y_pred)

print("Best Parameters:", grid.best_params_)
print("Recall setelah tuning:", round(recall, 2))

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
10 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
7 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/_p

Best Parameters: {'criterion': 'entropy', 'max_features': 'sqrt'}
Recall setelah tuning: 0.64


In [ ]:
import numpy as np

# =====================================
# 8. Analisis Feature Importance
# =====================================
importances = best_rf.feature_importances_
feature_names = X.columns

# Buat dataframe ranking
feat_imp = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Tampilkan semua feature importance
print("\n=== Feature Importance (Descending) ===")
print(feat_imp)

# Tampilkan 3 feature paling penting
top3 = feat_imp.head(3)
print("\nTop 3 fitur paling berpengaruh:")
print(top3)

# Tampilkan 2 feature paling lemah
bottom2 = feat_imp.tail(2)
print("\n2 fitur yang pengaruhnya paling kecil:")
print(bottom2)



=== Feature Importance (Descending) ===
                 Feature  Importance
1  glucose_concentration    0.260577
5                    bmi    0.165501
7                    age    0.147795
6      pedigree_function    0.107276
2         blood_pressure    0.096157
0           num_pregnant    0.082889
4       two_hour_insulin    0.070521
3      triceps_thickness    0.069284

Top 3 fitur paling berpengaruh:
                 Feature  Importance
1  glucose_concentration    0.260577
5                    bmi    0.165501
7                    age    0.147795

2 fitur yang pengaruhnya paling kecil:
             Feature  Importance
4   two_hour_insulin    0.070521
3  triceps_thickness    0.069284
